In [1]:
__author__ = "Dong Qichen"
__license__ = "GPL"
__version__ = "3.0"

In [2]:
%matplotlib widget

# Display the scalar potential for a point charge, for which the speed (in units of c), beta, can be varied 
# by means of a slider 

import matplotlib.pyplot as plt
from matplotlib import cm
from ipywidgets import AppLayout, FloatSlider
from ipywidgets import interact, interactive, fixed, interact_manual
import matplotlib.ticker as ticker
plt.ioff()

import numpy as np

In [3]:
# Set fundamental constants relevant to electrodynamics.
e = 1  # Elementary charge.
c = 1  # Speed of light.
h = 2 * np.pi  # Planck's constant.
alpha = 1/137  # Fine-structure constant.
epsilon_0 = e**2 / (2 * alpha * h * c)  # Vacuum permittivity.

In [4]:
def getGamma(beta):
    return 1 / (1 - beta**2)**0.5
def bremsstrahlungPower(theta, phi, beta, betaDot, q):
    return \
        epsilon_0 * c * q**2 / (16 * np.pi**2) \
        * np.sin(theta)**2 \
        * betaDot**2 / (1 - beta * np.cos(theta))**5

In [5]:
fig2d, ax2d = plt.subplots(subplot_kw={'projection': 'polar'})
theta = np.arange(0, 2*np.pi, np.pi / 200)
@interact(betaDot = (0.0, 2.0, 0.01), q = (0.1,5.0,0.1), qrestframe=True, beta = (0.0, 0.999, 0.001))
def plotBremsstrahlungPower2D(betaDot, q, beta=0, qrestframe=True): 
    frame = "charge rest frame" if qrestframe else "lab frame"
    power = bremsstrahlungPower(theta, 0, beta, betaDot, q)
    power = power / getGamma(beta)**6 if qrestframe else power
    ax2d.clear()
    ax2d.set_title(f"Bremsstrahlung Power, {frame}")
    ax2d.plot(theta, power)
    ax2d.set_rmax(2)
    ax2d.set_rticks([0, 0.5, 1.0, 1.5, 2])
    return fig2d

interactive(children=(FloatSlider(value=1.0, description='betaDot', max=2.0, step=0.01), FloatSlider(value=2.5…

The radiated bremsstrahlung power per unit solid angle is given by
  $$\frac{\mathrm{d}P_\parallel}{\mathrm{d}\Omega} = \frac{\mu_0 c q^2}{16\pi^2}\frac{\sin^2\theta\,\dot\beta^2}{\left(1-\beta\cos\theta\right)^5},$$
  where $\theta$ is the angle between $\boldsymbol{\beta}$ and $\boldsymbol{R}$

In [6]:
theta_mesh, phi_mesh = np.meshgrid(np.linspace(0, 2 * np.pi, 300), np.linspace(0, 2 * np.pi, 25))

fig3d = plt.figure(figsize=(10,10))
ax3d = fig3d.add_subplot(1,1,1, projection='3d')

@interact(betaDot=(0.0, 2.0, 0.01), q=(0.1,5.0,0.1), azim=(-90, 90, 1), roll=(-90, 90, 1), elev=(-90, 90, 1), beta=(0.0, 0.999, 0.001), qrestframe=True)
def plotBremsstrahlungPower3D(betaDot, q, azim, roll, elev, beta=0, qrestframe=True):
    power = bremsstrahlungPower(theta_mesh, phi_mesh, beta, betaDot, q)
    power = power / getGamma(beta)**6 if qrestframe else power
    power_X = power * np.sin(theta_mesh) * np.cos(phi_mesh)
    power_Y = power * np.sin(theta_mesh) * np.sin(phi_mesh)
    power_Z = power * np.cos(theta_mesh)
    ax3d.clear()
    ax3d.set_xlim(-2,2)
    ax3d.set_ylim(-2,2)
    ax3d.set_zlim(-2,2)
    ax3d.view_init(elev, azim, roll)
    my_color_map = cm.jet(power/np.amax(power))
    surface3d = ax3d.plot_surface(
        power_X, power_Y, power_Z, 
        facecolors=my_color_map,
        rstride=1, cstride=1, 
        linewidth=0, 
        antialiased=False, 
        alpha=0.1
    )
    return fig3d

interactive(children=(FloatSlider(value=1.0, description='betaDot', max=2.0, step=0.01), FloatSlider(value=2.5…